In [2]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora

In [67]:
table=pd.read_excel("mingpao.xlsx",names=["Date","Title","Part","Content","URL"])

In [22]:
np.savetxt("mingpao_text.txt",table["Content"], fmt='%s')

In [68]:
import jieba
import jieba.posseg as pseg
import operator

In [69]:
jieba.set_dictionary('C:\Users\junior\Anaconda2\Lib\site-packages\jieba\dict.txt.big')
jieba.load_userdict('C:\Users\junior\Anaconda2\Lib\site-packages\jieba\userdict-HKLeg.txt') 

Stoplist=[]

f2=open('./stopwords-HKLeg.txt')
for line in f2.readlines():
	line=line.strip('\n')
	Stoplist.append(line)

Building prefix dict from C:\Users\junior\Anaconda2\Lib\site-packages\jieba\dict.txt.big ...
Loading model from cache c:\users\junior\appdata\local\temp\jieba.ub4a9829ecdc747f8a94d4edc9928c7c5.cache
Loading model cost 1.114 seconds.
Prefix dict has been built succesfully.


In [70]:
word_dic={}
Text=[]
f=table["Content"]

for line in f:
    if type(line)!=float:
        try:
            words = list(jieba.cut(line))
            Text.append(words)
            for word in words:
                if (len(word)>1) and word not in Stoplist:#delete the words with only one character.
                    word_dic[word]=word_dic.get(word,0)+1
        except:
            print line

#delete words that only appear once
for key in word_dic.keys():
    if word_dic[key]==1:
        del word_dic[key]

#update the text
Text=[[n for n in i if n in word_dic.keys() ] for i in Text]

C:\Users\junior\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [82]:
dictionary = corpora.Dictionary(Text)
dictionary.save('./words.dict')
corpus = [dictionary.doc2bow(t) for t in Text]
corpora.MmCorpus.serialize('./corpus.mm', corpus)

In [3]:
dictionary=corpora.Dictionary.load('C:\Users\junior\jupyter\LDA-HDP\words2.dict')
corpus=corpora.MmCorpus('corpus2.mm')

In [7]:
for key in dictionary.token2id.keys():
    print key.encode("utf8"),"id:",dictionary.token2id[key]

搞點 id: 17204
第二 id: 3145
淵源 id: 16335
頭破血流 id: 13801
明確 id: 715
追隨 id: 10668
減債 id: 13364
起變 id: 20473
呼之則 id: 19026
行駛 id: 16262
張慧雯 id: 19996
困境 id: 997
這套 id: 15197
看似 id: 10611
阿果 id: 6315
為始 id: 12679
相對而言 id: 2514
以較 id: 8483
盈利 id: 15008
發問者 id: 4002
戴希立 id: 19520
至此 id: 2309
無財 id: 15487
揣摩 id: 17754
態度 id: 5
第一 id: 3176
周文慶 id: 15829
無責 id: 1250
第三 id: 404
漸漸 id: 5112
小至 id: 17640
抱住 id: 525
桑德斯 id: 3673
公路 id: 4327
政策方針 id: 1606
Hamilton id: 2526
讓位 id: 13979
音樂創作 id: 11441
實事 id: 2527
資格賽 id: 21701
戰機 id: 21628
升跌 id: 18187
看來 id: 2533
為奇 id: 18151
保護林 id: 16694
民主黨員 id: 11592
身段 id: 2539
工聯會 id: 286
凍產 id: 12824
買不到 id: 13753
曾健成 id: 8704
零息 id: 20908
兼顧 id: 3223
悲劇 id: 10260
取得成功 id: 6269
亂糟糟 id: 9770
獻禮 id: 5898
建成 id: 5370
查問 id: 2128
猛人 id: 19512
押送 id: 16646
另有 id: 5779
這場 id: 2561
韓聯社 id: 6274
次季 id: 20513
這塊 id: 11786
澳紐 id: 18259
空管 id: 6051
看中 id: 20591
強於 id: 22772
大刀闊斧 id: 14992
由淺入深 id: 13065
真可 id: 22686
次子 id: 15715
不寒而慄 id: 16429
內訌 id: 1305
hk id: 7522
集結 id

ValueError: I/O operation on closed file

In [109]:
import gensim
hdp = gensim.models.hdpmodel.HdpModel(corpus, dictionary,kappa=0.5,chunksize=256)

In [110]:
topics=hdp.show_topics(topics=-1, topn=30)
print len(topics)

150


In [111]:
lda= hdp.hdp_to_lda()

In [112]:
lda[0][lda[0]>0.01]

array([ 0.66871483,  0.08032153,  0.02759499,  0.02224028,  0.01517832,
        0.01127468,  0.01143516])

In [113]:
print topics[0]

topic 0: 0.012*選舉 + 0.010*香港 + 0.008*參選 + 0.007*立法會 + 0.005*報道 + 0.005*港獨 + 0.004*支持 + 0.004*政治 + 0.004*參選人 + 0.004*確認書 + 0.004*提名 + 0.003*政府 + 0.003*議員 + 0.003*特首 + 0.003*基本法 + 0.003*問題 + 0.003*名單 + 0.003*主席 + 0.003*泛民 + 0.003*要求 + 0.002*主任 + 0.002*簽署 + 0.002*梁振英 + 0.002*報名 + 0.002*可能 + 0.002*中國 + 0.002*候選人 + 0.002*社會 + 0.002*獨立 + 0.002*美國


In [115]:
f=open('topics-output2.txt','wb')
new=np.asarray(topics)[lda[0]>0.01]
for i in range(len(new)):
    f.write(new[i].encode("utf8")+'\n')

In [4]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20, update_every=1, chunksize=256, passes=1)

In [124]:
f3=open('./LDA-doc-topic.txt','w')
n=0
for line in corpus:
	topiclist=lda.get_document_topics(line)
	corpus[n].append(topiclist[0][0])
	f3.write('%d,%d,%f\n'%(n,topiclist[0][0],topiclist[0][1]))
	n=n+1
f3.close

<function close>

In [5]:
#print topic-words
f4=open('./LDA-topic-words.txt','w')
word2id=dictionary.token2id
word2id= sorted(word2id.iteritems(), key=lambda d:d[1], reverse = False) #from small to large
words=[i[0] for i in word2id]
for i in range(10):
    wordlist=lda.get_topic_terms(i,topn=30)
    liststring=''
    for n in range(30):
        Id=wordlist[n][0]
        word=words[Id]
        liststring=liststring+','+word
    f4.write('%d%s\n'%(i,liststring.encode("utf8")))
    print '%d%s\n'%(i,liststring.encode("utf8"))

0,提名,參選人,選舉,九龍,選管會,委會,新政,新界,議席,決定,委員會,候選人,區議會,新東,要求,表格,今年,強調,維持,不變,確認書,下屆,宣布,報告,感到,代表,改革,大會,正式,人數

1,執行,希望,工作,覺得,議會,擔心,工黨,接受,很多,增加,朋友,之間,問到,訪問,真的,關注,英文,政府,大局,改善,表現,機會,人選,生活,集會,提醒,聲音,選擇,整體,年來

2,醫生,確認,通過,界別,挑戰,部分,現任,聯盟,業界,明確,表明,註冊,鄭錦滿,理由,成員,機構,來自,大選,文摘,病人,工程,政府,委員會,歐盟,傳媒,發生,正義,執政,補充,復興

3,聲明,of,希拉里,for,that,符合,公司,加息,繼續,資助,離職,自覺,and,本報,員工,投資者,擁有,美國,方法,財經,新聞,虛假,組織,com,同事,by,資產,as,鼓勵,雜誌

4,基本法,確認書,法律,港人,要求,獨立,香港,有效,內地,管治,人士,管理,司長,行動,社會,律師,成為,效忠,民主,曾經,當局,必須,政府,刑事,發言人,多個,一旦,公平,政治,少數

5,事件,人民,日本,社民連,召集人,程序,梁頌恆,協助,人員,萬元,短片,推出,吳文遠,規定,涉嫌,周浩鼎,首次,10,目標,被捕,計劃,修改,16,打壓,時間,提供,拘捕,楊岳橋,存在,控方

6,選舉,參選,立法會,特首,議員,梁振英,報名,支持,曾鈺成,主席,主任,泛民,連任,民建聯,建制派,民主黨,獨立,公民,候選人,考慮,表態,出選,工聯會,今日,政府,要求,政黨,CY,港島,未有

7,to,支持者,is,方面,independence,投資,be,have,will,開始,it,於是,election,people,特別,they,政治立場,台灣,突然,呼籲,活動,their,from,人物,只要,government,declaration,參與,傾向,網站

8,香港,港獨,中國,問題,一國兩制,青年,自決,2047,政治,需要,市民,可能,處理,強調,權利,基本法,這些,本地,影響,利益,相信,這種,考慮,自決權,矛盾,法官,不能,港台,回歸,目的

9,部分,條文,聯辦,運動,進入,學生,記者會,而是,清楚,知道,現在,不可,不少,明日,絕不,完全,權力,情緒,政治,歷史,造成,遊戲,這樣,官方